# Import and Load "Database" into memory

In [2]:
from pathlib import Path
import os
from dotenv import load_dotenv

load_dotenv()

DB_PATH = Path(str(os.getenv('DB_PATH')))

import numpy as np
import pandas as pd
import geopandas as gpd

document_files = pd.read_parquet(DB_PATH / "document_files.parquet")
locations = gpd.read_parquet(DB_PATH / "locations.parquet")
project_total_scopes = pd.read_parquet(DB_PATH / "project_total_scopes.parquet")
project_safety_scopes = pd.read_parquet(DB_PATH / "project_safety_scopes.parquet")
project_geos = gpd.read_parquet(DB_PATH / "project_geos.parquet")
projects = pd.read_parquet(DB_PATH / "projects.parquet")
location_year_files = pd.read_parquet(DB_PATH / "location_year_files.parquet")
document_collections = pd.read_parquet(DB_PATH / "document_collections.parquet")
location_to_project = pd.read_parquet(DB_PATH / "_location_to_project.parquet")
imgs_to_filesAPI = pd.read_parquet(DB_PATH / '_imgs_to_filesAPI.parquet')
docs_to_filesAPI = pd.read_parquet(DB_PATH / '_docs_to_filesAPI.parquet')
image_pairs = pd.read_parquet(DB_PATH / 'processed' / 'image_pairs.parquet')

# Image Change Identifier

In [64]:
image_pairs.merge(
    location_to_project,
    how='left',
    left_on='location_id', right_on='location_id',
    indicator=True
)['_merge'].value_counts() # so 2986 don't go to a project. That is less than I thought?

# Let's check on here
locations.merge(
    location_to_project,
    how='left',
    left_on='location_id', right_on='location_id',
    indicator=True
)['_merge'].value_counts() # ok 299/2000. I guess its cause there are a lot of duplicates in `location_to_project`

# Now back to the merge
merged_imgpairs_to_projects = image_pairs.merge(
    location_to_project,
    how='left',
    left_on='location_id', right_on='location_id'
).merge(
    projects,
    how='left',
    left_on='citydata_proj_id', right_on='citydata_proj_id'
)

# Mask - features
has_safety = merged_imgpairs_to_projects['safety_scope'].notna()
has_total = merged_imgpairs_to_projects['total_scope'].notna()

# Masks - Proj_year between start and end years
proj_before_end = merged_imgpairs_to_projects['proj_year'] < merged_imgpairs_to_projects['year_end'].astype(int)
proj_after_start = merged_imgpairs_to_projects['proj_year'] > merged_imgpairs_to_projects['year_start'].astype(int)

# Mask - Remove those where we can't tell because the project year == end or start
proj_equals_end = merged_imgpairs_to_projects['proj_year'] == merged_imgpairs_to_projects['year_end'].astype(int)
proj_equals_start = merged_imgpairs_to_projects['proj_year'] == merged_imgpairs_to_projects['year_start'].astype(int)

# Mask - missing
year_missing = merged_imgpairs_to_projects['proj_year'].isna()

# Total_scope Class
merged_imgpairs_to_projects.loc[has_total & proj_before_end & proj_after_start, 'change_identifier_total'] = 'Change Between'
merged_imgpairs_to_projects.loc[has_total & (~proj_before_end | ~proj_after_start), 'change_identifier_total'] = 'Change Out of Range'
merged_imgpairs_to_projects.loc[has_total & (proj_equals_start | proj_equals_end), 'change_identifier_total'] = 'Change On Year'
merged_imgpairs_to_projects.loc[~has_total | year_missing, 'change_identifier_total'] = 'No Change'

# Total_scope Class
merged_imgpairs_to_projects.loc[has_safety & proj_before_end & proj_after_start, 'change_identifier_safety'] = 'Change Between'
merged_imgpairs_to_projects.loc[has_safety & (~proj_before_end | ~proj_after_start), 'change_identifier_safety'] = 'Change Out of Range'
merged_imgpairs_to_projects.loc[has_safety & (proj_equals_start | proj_equals_end), 'change_identifier_safety'] = 'Change On Year'
merged_imgpairs_to_projects.loc[~has_safety | year_missing, 'change_identifier_safety'] = 'No Change'


#merged_imgpairs_to_projects[['location_id', 'year_start','year_end','image_change']]
merged_imgpairs_to_projects['safety_scope'].isna().sum() # Ok so 9k/20k have no safety scope. GOOD to know
merged_imgpairs_to_projects['total_scope'].isna().sum() # 3k have no scope

merged_imgpairs_to_projects[merged_imgpairs_to_projects['total_scope'].isna()]['proj_year'].value_counts(dropna=False) # Ok so the vast majority of them are straight up missing but some aren't

# We can determine class by the sample or by the results


proj_year
NaN       2966
2005.0     370
2015.0     200
2019.0      80
2009.0      10
2013.0      10
Name: count, dtype: int64

In [ ]:
merged_imgpairs_to_projects
sample = pd.read_csv('../data_oai_files/samples/image_pairs.csv')
ID_COLS = ['location_id', 'year_start', 'year_end']
sample[ID_COLS] = sample['item_id'].str.split('-', expand=True)
sample['location_id'] = sample['location_id'].astype(int)

# sample.merge(
#     locations,
#     on='location_id'
# )['sample_source'].value_counts()

# merged_imgpairs_to_projects.merge(
#     locations,
#     on='location_id'
# )['sample_source'].value_counts()

sample_source
Safety              24580
noSafety            11470
postIntervention     4004
none                 3076
Name: count, dtype: int64

In [60]:
merged_imgpairs_to_projects['change_identifier_safety'].value_counts()

#merged_imgpairs_to_projects['change_identifier_total'].value_counts()

change_identifier_safety
No Change              17726
Change Between         12094
Change Out of Range    11918
Change On Year          1392
Name: count, dtype: int64

In [ ]:
#merged_imgpairs_to_projects.drop(['imagery_path_start','imagery_path_end','uri_start','uri_end','file_name_start','file_name_end','index_right','projID_dontuse','ProjTitle','LeadAgency',''])

gt_df = merged_imgpairs_to_projects[['location_id','year_start', 'year_end','citydata_proj_id', 'proj_year','change_identifier_total','change_identifier_safety', 'total_scope','safety_scope']]
gt_df_deduped = gt_df.drop_duplicates(['location_id','year_start', 'year_end','citydata_proj_id', 'proj_year','change_identifier_total','change_identifier_safety'])

# Create Safety_scope_plus - just add Sidewalks
total_scop_expl = gt_df_deduped['total_scope'].explode() #['safety_scope_plus']
total_scop_expl_nona = total_scop_expl.where(~total_scop_expl.isin(['Partial Reconstruction', 'Curb to Curb Reconstruction', 'Resurfacing'])).dropna()
total_scop_expl_nona_grpd = total_scop_expl_nona.groupby(level=0).agg(list).rename('safety_scope_plus')
gt_df_deduped = pd.concat([gt_df_deduped, total_scop_expl_nona_grpd],axis=1, join='inner')

#gt_df_deduped.to_parquet(DB_PATH / 'groundtruth' / 'location_pair_change_identifier.parquet')

# gt_df_deduped.merge(
#     locations,
#     on='location_id',
# ).drop_duplicates(subset=['location_id','sample_source','change_identifier_total'])[['sample_source', 'change_identifier_total']].value_counts().unstack()

sample.drop_duplicates(subset=['location_id']).merge(
    locations,
    on='location_id'
)['sample_source'].value_counts()

sample_source
Safety              830
noSafety             88
none                 53
postIntervention     29
Name: count, dtype: int64

In [ ]:
locations.merge(
    location_to_project,
    on = 'location_id',
    how='left', indicator='loc_to_proj'
#)['sample_source'].value_counts() # 308 none, 404, postintervention
#).drop_duplicates(subset=['location_id','sample_source'])['sample_source'].value_counts() # 300 none, 132 postintervention
#)['loc_to_proj'].value_counts() #4020, 297
).merge(
    projects,
    on='citydata_proj_id',
    how='left', indicator='l2p_to_proj'
#)[['loc_to_proj','l2p_to_proj']].value_counts().unstack() # 4020l, 297
)[]

,location_id,crossstreets,geometry,sample_source,citydata_proj_id,index_right,distance,loc_to_proj,projID_dontuse,ProjTitle,LeadAgency,ProjectType,ProjectStatus,proj_year,safety_scope,total_scope,l2p_to_proj
0,43609,"[W 182 ST, WADSWORTH AVE]",POINT (-73.93454 40.85042),Safety,2836.0,1935.0,0.179325,both,3082.0,Safe Routes to Schools,DOT,CAPITAL RECONSTRUCTION,Completed Project,2015.0,"[Raised Median, Bus Bulb, Curb Extensions]","[Partial Reconstruction, Raised Median, Bus Bu...",both
1,41675,"[30 AVE, 29 ST]",POINT (-73.92298 40.76764),Safety,315.0,199.0,177.099527,both,868.0,School Safety,DOT,CAPITAL RECONSTRUCTION,Completed Project,2016.0,"[Median Tip Extension, Bus Bulb, Curb Extensions]","[Partial Reconstruction, Median Tip Extension,...",both
2,41675,"[30 AVE, 29 ST]",POINT (-73.92298 40.76764),Safety,315.0,199.0,0.249511,both,868.0,School Safety,DOT,CAPITAL RECONSTRUCTION,Completed Project,2016.0,"[Median Tip Extension, Bus Bulb, Curb Extensions]","[Partial Reconstruction, Median Tip Extension,...",both
3,37511,"[182 PL, 89 AVE]",POINT (-73.77928 40.71225),Safety,2860.0,1959.0,0.292503,both,3083.0,Safe Routes to Schools,DOT,CAPITAL RECONSTRUCTION,Completed Project,2013.0,"[Median Tip Extension, Curb Extensions]","[Partial Reconstruction, Median Tip Extension,...",both
4,54810,"[71 RD, 136 ST]",POINT (-73.82715 40.72605),Safety,1398.0,973.0,0.267454,both,1233.0,Pedestrian Safety Non-Federal,DOT,CAPITAL RECONSTRUCTION,Completed Project,2013.0,"[Bus Bulb, Curb Extensions]","[Bus Bulb, Curb Extensions]",both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4312,36332,"[ROCKAWAY BLVD, 149 ST]",POINT (-73.78902 40.67316),postIntervention,2005.0,1432.0,0.280786,both,1352.0,"Sutter Avenue, etc Phase II",DOT,CAPITAL RECONSTRUCTION,Completed Project,2005.0,None,"[Curb to Curb Reconstruction, Sidewalks]",both
4313,9061704,[STEINWAY ST],POINT (-73.91031 40.76937),postIntervention,2785.0,1884.0,28.946423,both,3068.0,Steinway ST BR 2-23061 GCP 2-23061-0,DOT,CAPITAL RECONSTRUCTION,Completed Project,2002.0,None,"[Curb to Curb Reconstruction, Sidewalks]",both
4314,9061704,[STEINWAY ST],POINT (-73.91031 40.76937),postIntervention,2785.0,1884.0,256.647483,both,3068.0,Steinway ST BR 2-23061 GCP 2-23061-0,DOT,CAPITAL RECONSTRUCTION,Completed Project,2002.0,None,"[Curb to Curb Reconstruction, Sidewalks]",both
4315,9061704,[STEINWAY ST],POINT (-73.91031 40.76937),postIntervention,86.0,26.0,28.946423,both,667.0,STEINWAY ST BR 2-23060 BQE SB,DOT,CAPITAL RECONSTRUCTION,Completed Project,2002.0,None,"[Curb to Curb Reconstruction, Sidewalks]",both


In [ ]:
(
    merged_imgpairs_to_projects.merge(
    #merged_imgpairs_to_projects.merge(
        locations[['location_id','sample_source']],
        left_on='location_id', right_on='location_id',
        how = 'left'
    ).groupby('sample_source')['change_identifier_safety'].value_counts()
    .unstack()
    .style
        .background_gradient(cmap='Greens')
        #.format('{:.1%}')
        .format('{:1}')
) # Looks Good on both total and safety. 
# the two look similar but way more noSafety having Change with total rather than safety as expected
# 'None' is all No Change as hoped
# 'postIntervention' has some Change Between, I assume because of redos

(
    merged_imgpairs_to_projects
    .groupby(['year_start','year_end'])['change_identifier_total']
    .value_counts(normalize=True)
    .unstack()
    .apply(np.round, decimals=3)
    .style
        .background_gradient(cmap='Greens')
        .format('{:.1%}')
) # Looks great!

(
    merged_imgpairs_to_projects
    .groupby(['year_start','year_end'])['change_identifier_safety']
    .value_counts(normalize=True)
    .unstack()
    .apply(np.round, decimals=3)
    .style
        .background_gradient(cmap='Blues')
        .format('{:.1%}')
) # Same!

### Sampling

In [203]:
# So I can't use sample for my classifier because I need the specific (l_id, before, after) pairs
# But I can use it for a shortcut to downsample beforehand
#locations['sample'].isin(['Safety', 'postIntervention', 'noSafety'])

# So for sampling:
# Let's first grab some from the Safety subset

# let's just sample from the tagged merged pairs 
# - we'll ignore change on Year (for simplicity)
# - but oversample from Change between and remove
merged_imgpairs_to_projects['change_identifier_safety'].value_counts() # so like half No Change and ~25% Change Between
#merged_imgpairs_to_projects['change_identifier_total'].value_counts() # Now like 40% No Change. almost 40% Change Between

# Let's just grab 750 pairs for Change Between for safety
# 150 for No Change
# 100 for Change Outside Range

# # Hmm I actually want some of the same locations.
SAMPLE_N_SIZE = 2000
CLASS_N_PROPORTION = {
    'Change Between': .75,
    'Change Out of Range': .1,
    'No Change': .15,
    'Change On Year': 0 # Ignore
}

CLASS_COL = 'change_identifier_safety'
class_samples = {}

ID_COLS = ['location_id', 'year_start', 'year_end', CLASS_COL, 'change_identifier_total']
for c in CLASS_N_PROPORTION:
    class_samples[c] = merged_imgpairs_to_projects[merged_imgpairs_to_projects[CLASS_COL] == c].sample(int(SAMPLE_N_SIZE * CLASS_N_PROPORTION[c]))

change_identifier_classes = pd.concat(class_samples, ignore_index=True)[ID_COLS]
change_identifier_classes = change_identifier_classes.sample(SAMPLE_N_SIZE)


change_identifier_classes['location_id'].value_counts()

location_id
19022      8
16112      7
15733      7
90258      7
27885      7
          ..
83954      1
35393      1
9006972    1
20695      1
78199      1
Name: count, Length: 1015, dtype: int64

NameError: name 'change_identifier_classes' is not defined

In [ ]:
#change_identifier_classes.to_parquet(DB_PATH / 'groundtruth' / 'change_identifier_classes.parquet')

In [134]:
# Ok so lets just load in the results and see what happens
from dataclasses import fields
from streettransformer.alignment.validate import set_up_validation_dataframe, ChangeIdentifierKey, ChangeIdentifierOutput
OAI_RESULTS = Path('../data_oai_files/results')
response_df = pd.read_json(OAI_RESULTS / 'identifier-image-4o.csv', lines=True)
results_df = response_df['output_text'].apply(pd.Series)
results_cols = [f.name for f in fields(ChangeIdentifierOutput)]

id_cols = [f.name for f in fields(ChangeIdentifierKey)]

unified_response_df = pd.concat([
    response_df['item_id'].str.split('-', expand=True).set_axis(id_cols, axis=1),
    response_df['model'],
    results_df[results_cols]
], axis=1, join='inner')
unified_response_df['location_id'] = unified_response_df['location_id'].astype(int)



joined_to_GT = unified_response_df.merge(
    location_to_project,
    on = 'location_id',
    how='left'
).merge(
    projects,
    on = 'citydata_proj_id',
    how='left'
).drop_duplicates(subset=['location_id', 'year_start','year_end', 'citydata_proj_id'])


# Class for Total Change

# Change Between
mask_after_start = joined_to_GT['year_start'].astype(int) < joined_to_GT['proj_year']
mask_before_end = joined_to_GT['year_end'].astype(int) > joined_to_GT['proj_year']
joined_to_GT.loc[(mask_after_start & mask_before_end), 'project_range'] = 'Between'

# No Change
mask_no_change = joined_to_GT['proj_year'].isna()
joined_to_GT.loc[mask_no_change, 'project_range'] = 'None'

# Change On Year
mask_change_year_start = joined_to_GT['year_start'].astype('int') == joined_to_GT['proj_year']
mask_change_year_end = joined_to_GT['year_end'].astype('int') == joined_to_GT['proj_year']
joined_to_GT.loc[(mask_change_year_start | mask_change_year_end), 'project_range'] = 'On Year'

# Change Out of Range
mask_before_start = joined_to_GT['year_start'].astype('int') > joined_to_GT['proj_year']
mask_after_end = joined_to_GT['year_end'].astype('int') < joined_to_GT['proj_year']
joined_to_GT.loc[(mask_before_start | mask_after_end), 'project_range'] = 'Outside'

#results_df[results_cols
joined_to_GT['project_range'].value_counts()


project_range
Between    1068
Outside     247
None         52
On Year      38
Name: count, dtype: int64

In [155]:
import numpy as np
joined_to_GT[joined_to_GT['safety_scope'].isna()]['total_scope'].value_counts()


# Pos class definition:
mask_safety_scope = joined_to_GT['safety_scope'].notna()
mask_sidewalk = 'Sidewalks' in joined_to_GT['total_scope']
mask_sidewalk_plaza = 'Plaza/Ped Space Enhancement' in joined_to_GT['total_scope']

mask_change_defined = (mask_safety_scope | mask_sidewalk | mask_sidewalk_plaza)

joined_to_GT['class'] = False
mask_between = (joined_to_GT['project_range'] == 'Between')
mask_on_year = (joined_to_GT['project_range'] == 'On Year')

joined_to_GT.loc[(mask_change_defined & mask_between), 'class'] = True
joined_to_GT.loc[(mask_change_defined & mask_on_year), 'class'] = None
# joined_to_GT[mask_change_defined & (joined_to_GT['project_range'] == 'On Year'), 'class'] = None
# #joined_to_GT[mask_change_defined & (joined_to_GT['project_range'] == 'Outside'), 'class'] = False
# #joined_to_GT[mask_change_defined & (joined_to_GT['project_range'] == 'None'), 'class'] = False

/var/folders/t8/dm9l8xy95mv0d_75b2m8r5nw0000gn/T/ipykernel_56128/3781798326.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  joined_to_GT.loc[(mask_change_defined & mask_on_year), 'class'] = None


In [ ]:
joined_to_GT['class'].value_counts()

joined_to_GT

class
True     983
False    406
Name: count, dtype: int64

In [ ]:
joined_to_GT[['location_id', 'year_start','year_end', 'class']].drop_duplicates()

#joined_to_GT.groupby(['location_id', 'year_start','year_end'])['class'].agg(sum)
change_classifier_class = joined_to_GT.groupby(['location_id', 'year_start','year_end'])['class'].agg('any')
change_classifier_class.to_csv(DB_PATH / 'groundtruth' / 'change_classifier_class.csv')


# Now again for Identifier

In [ ]:
total_scop_expl = joined_to_GT[ID_COLS + ['total_scope', 'project_range']].explode('total_scope')
INVALID_SCOPES = ['Partial Reconstruction', 'Curb to Curb Reconstruction', 'Resurfacing']
total_scop_valid = total_scop_expl[~total_scop_expl['total_scope'].isin(INVALID_SCOPES)]
total_scop_valid # 5,928 --> 4,419

mask_between = (total_scop_valid['project_range'] == 'Between')
mask_on_year = (total_scop_valid['project_range'] == 'On Year')

#total_scop_valid_deduped = total_scop_valid.drop_duplicates()
#total_scop_valid['total_scope_list'] = total_scop_valid.groupby(level=0)['total_scope'].agg(list)

total_scop_valid.groupby(ID_COLS + ['project_range'])['total_scope'].agg(lambda x: list(set(x))).reset_index()
ignore_on_year = total_scop_valid[~mask_on_year].groupby(ID_COLS + ['project_range'])['total_scope'].agg(lambda x: list(set(x))).reset_index()
# Just remove on year

ignore_on_year[ignore_on_year.duplicated(subset=ID_COLS)] # all duplicated are Outside

mask_between = ignore_on_year['project_range'] == 'Between'
ignore_on_year.loc[mask_between, 'correct_tags'] = ignore_on_year.loc[mask_between, 'total_scope']
#ignore_on_year.loc[~mask_between, 'correct_tags'] = np.array([[] for _ in range((~mask_between).sum())])
#ignore_on_year['correct_tags'].fillna(value
ignore_on_year['correct_tags']=ignore_on_year['correct_tags'].apply(lambda x: x if isinstance(x, list) else [])

#ignore_on_year.drop_duplicates(subset=['location_id', 'year_start','year_end','project_range'])
feature_tagger_GT = ignore_on_year.sort_values('project_range').drop_duplicates(subset=ID_COLS).sort_index().drop('total_scope', axis=1)

feature_tagger_GT.to_parquet(DB_PATH / 'groundtruth/change_identifier.parquet')

# If project_range == None, --> None


In [ ]:
# ## Now need to redo for change not including Just Resurfacing
# #merged_imgpairs_to_projects.drop(['imagery_path_start','imagery_path_end','uri_start','uri_end','file_name_start','file_name_end','index_right','projID_dontuse','ProjTitle','LeadAgency',''])

# gt_df = merged_imgpairs_to_projects[['location_id','year_start', 'year_end','citydata_proj_id', 'proj_year','change_identifier_total','change_identifier_safety', 'total_scope','safety_scope']]
# gt_df_deduped = gt_df.drop_duplicates(['location_id','year_start', 'year_end','citydata_proj_id', 'proj_year','change_identifier_total','change_identifier_safety'])

# # Create Safety_scope_plus - just add Sidewalks
# total_scop_expl = gt_df_deduped['total_scope'].explode() #['safety_scope_plus']
# total_scop_expl_nona = total_scop_expl.where(~total_scop_expl.isin(['Partial Reconstruction', 'Curb to Curb Reconstruction', 'Resurfacing'])).dropna()
# total_scop_expl_nona_grpd = total_scop_expl_nona.groupby(level=0).agg(list).rename('safety_scope_plus')
# gt_df_deduped = pd.concat([gt_df_deduped, total_scop_expl_nona_grpd],axis=1, join='inner')


In [311]:
import math
projects_copy = projects.copy()
projects_copy['proj_year'] = projects_copy['proj_year'].fillna(0).astype(int).astype(str).replace('0', pd.NA)

location_to_proj_year = locations.merge(
    location_to_project,
    on = 'location_id',
    how = 'left'
).merge(
    projects_copy,
    on = 'citydata_proj_id',
    how = 'left'
).groupby('location_id')['proj_year'].agg(lambda x: list(set(x)))#.value_counts()




In [314]:
location_to_proj_year.reset_index().to_parquet('../db/groundtruth/dater.parquet')

location_id
72          [nan]
604         [nan]
1514        [nan]
2258       [2012]
2299        [nan]
            ...  
9066196    [2014]
9066199    [2014]
9066221    [2019]
9066224    [2019]
9066225    [2019]
Name: proj_year, Length: 1678, dtype: object